# Данные по тенисным матчам WTA 1969-2016

In [96]:
import numpy as np
import pandas as pd

fields = ['surface', 'winner_id', 'loser_id', 'winner_name', 'loser_name', 'winner_hand',
          'loser_hand','winner_ht', 'loser_ht', 'winner_ioc', 'loser_ioc', 'winner_age', 'loser_age']
df = pd.read_csv('wta_matches_1969.csv', usecols=fields)
for i in xrange(1970, 2017):
    fn = 'wta_matches_{0:d}.csv'.format(i)
    df_ = pd.read_csv(fn, usecols=fields)
    df = pd.concat([df, df_], ignore_index=True)
df

,surface,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age
0,Grass,200190,Billie Jean King,R,NaN,USA,NaN,200191,Frances Luff,U,NaN,AUS,NaN
1,Grass,200150,Evonne Cawley,R,NaN,AUS,NaN,200192,Helen Kayser,U,NaN,AUS,NaN
2,Grass,200168,Karen Krantzcke,R,NaN,AUS,NaN,200155,Lexie Kenny,U,NaN,AUS,NaN
3,Grass,200149,Helen Sheedy,U,NaN,AUS,NaN,200165,Lany Kaligis,U,NaN,INA,NaN
4,Grass,200193,Ann Jones,L,NaN,GBR,NaN,200166,Patricia Edwards,U,NaN,AUS,NaN
5,Grass,200145,Kerry Harris,U,NaN,AUS,NaN,200179,Barbara Hawcroft,U,NaN,AUS,NaN
6,Grass,200185,Lesley Hunt,U,NaN,AUS,NaN,200161,Janine Whyte,U,NaN,AUS,NaN
7,Grass,200129,Lesley Bowrey,R,NaN,AUS,NaN,200141,Wendy Gilchrist,L,NaN,AUS,NaN
8,Grass,200194,Francoise Durr,R,NaN,FRA,NaN,200195,H Rush,U,NaN,AUS,NaN
9,Grass,200184,Helen Cawley,U,NaN,AUS,NaN,200160,Judy Dalton,U,NaN,AUS,NaN


### Гипотеза о том, что правше сложнее играть с левшой:

In [237]:
RWinnerVsR = df[(df.winner_hand=='R') & (df.loser_hand=='R')][['winner_id']]\
.groupby(['winner_id']).agg({'winner_id':['count']})
RWinnerVsR.columns = RWinnerVsR.columns.droplevel(0)

RLoserVsR = df[(df.loser_hand=='R') & (df.winner_hand=='R')][['loser_id']]\
.groupby(['loser_id']).agg({'loser_id':['count']})
RLoserVsR.columns = RLoserVsR.columns.droplevel(0)

RWinnerVsL = df[(df.winner_hand=='R') & (df.loser_hand=='L')][['winner_id']]\
.groupby(['winner_id']).agg({'winner_id':['count']})
RWinnerVsL.columns = RWinnerVsL.columns.droplevel(0)

RLoserVsL = df[(df.loser_hand=='R') & (df.winner_hand=='L')][['loser_id']]\
.groupby(['loser_id']).agg({'loser_id':['count']})
RLoserVsL.columns = RLoserVsL.columns.droplevel(0)

df2 = pd.merge(RWinnerVsR, RLoserVsR, left_index=True, right_index=True, suffixes=['_winVsR', '_loseVsR'])

df2 = pd.merge(df2, RWinnerVsL, left_index=True, right_index=True)
df2 = df2.rename(columns={'count': 'count_winVsL'})

df2 = pd.merge(df2, RLoserVsL, left_index=True, right_index=True)
df2 = df2.rename(columns={'count': 'count_loseVsL'})

df2['count_VsR'] = df2['count_winVsR'] + df2['count_loseVsR']
df2['count_VsL'] = df2['count_winVsL'] + df2['count_loseVsL']
df2['winVsR %'] = 100 * df2['count_winVsR'] / df2['count_VsR']
df2['winVsL %'] = 100 * df2['count_winVsL'] / df2['count_VsL']
df2['Left-hander is a more difficult opponent'] = df2['winVsL %'] < df2['winVsR %']

print 'Left-hander is a more difficult opponent for {0:d} players.'.format(
    df2[df2['Left-hander is a more difficult opponent']==True].shape[0])
print 'Right-hander is a more difficult opponent for {0:d} players.'.format(
    df2[df2['Left-hander is a more difficult opponent']==False].shape[0])

Left-hander is a more difficult opponent for 234 players.
Right-hander is a more difficult opponent for 253 players.


- Вывод: похоже "леворукость" не преимущество :)

### Гипотеза о том, что рост в теннисе - преимущество

In [256]:
s = df['winner_ht'] - df['loser_ht']
s = s[~np.isnan(s)]
print 'Побед при превосходстве над соперноком в росте: {0:d}.'.format(s[(s > 0)].shape[0])
print 'Побед при превосходстве сопернока в росте: {0:d}.'.format(s[(s < 0)].shape[0])
print 

Побед при превосходстве над соперноком в росте: 10851.
Побед при превосходстве сопернока в росте: 9183.


In [262]:
s = df['winner_ht'] - df['loser_ht']
s = s[(~np.isnan(s)) & (np.abs(s)>=5)]
print 'Побед при превосходстве над соперноком в росте на 5 и более см: {0:d}.'.format(s[(s > 0)].shape[0])
print 'Побед при превосходстве сопернока в росте на 5 и более см: {0:d}.'.format(s[(s < 0)].shape[0])

Побед при превосходстве над соперноком в росте на 5 и более см: 7590.
Побед при превосходстве сопернока в росте на 5 и более см: 6152.


In [263]:
s = df['winner_ht'] - df['loser_ht']
s = s[(~np.isnan(s)) & (np.abs(s)>=10)]
print 'Побед при превосходстве над соперноком в росте на 10 и более см: {0:d}.'.format(s[(s > 0)].shape[0])
print 'Побед при превосходстве сопернока в росте на 10 и более см: {0:d}.'.format(s[(s < 0)].shape[0])

Побед при превосходстве над соперноком в росте на 10 и более см: 4009.
Побед при превосходстве сопернока в росте на 10 и более см: 3008.


In [264]:
s = df['winner_ht'] - df['loser_ht']
s = s[(~np.isnan(s)) & (np.abs(s)>=15)]
print 'Побед при превосходстве над соперноком в росте на 15 и более см: {0:d}.'.format(s[(s > 0)].shape[0])
print 'Побед при превосходстве сопернока в росте на 15 и более см: {0:d}.'.format(s[(s < 0)].shape[0])

Побед при превосходстве над соперноком в росте на 15 и более см: 1678.
Побед при превосходстве сопернока в росте на 15 и более см: 1087.


- Гипотеза скорее верна